## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [15]:
import numpy as np
import pandas as pd

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 69

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯ'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [16]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('../kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [25]:
import nltk
from nltk import word_tokenize

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

def find_max_str(companies):
    max_str = ''
    for company in companies:
        for word in word_tokenize(company):
            if(has_letters(word)):
                max_str = word if len(max_str) < len(word) else max_str 
    return max_str

def find_index_of_substring(string, companies):
    for i in range(len(companies)):
        if(companies[i].find(string) != -1):
            return i

В данной строке артефакт, подробнее в 'Testing.ipynb'

In [18]:
del train_companies[471970]

In [23]:
def extract_features(company_name, test=False):
    
    # Длина слова, количество гласных/согласных, положение в слове,
    # есть ли в слове что-то помимо букв и то же самое для предыдущего слова
    num_of_features = 10
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            if(index > 0):
                features[index][:5] = features[index - 1][5:]
            features[index][0] = index/size
            features[index][1] = len(word)/MAX_WORD
            features[index][2] = count_consonants(word)/len(word)
            features[index][3] = count_vowels(word)/len(word)
            features[index][4] = int(has_not_letters(word))
            if(word[0].isupper()):
                if(word[-1].isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return features, labels
    else:
        return features

In [24]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features = np.concatenate((features, curr_features))
            labels = np.concatenate((labels, curr_labels))
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [21]:
def transform(company_name, labels):
    index = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            if(labels[index] == 0):
                print(word.lower())
            elif(labels[index] == 1):
                print(word[0] + word[1:].lower())
            else:
                print(word)
            index += 1

In [48]:
X_train, y_train = generate_dataset(train_companies[:100000])

In [49]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier()
clf.fit(X_train, y_train)

Learning rate set to 0.109703
0:	learn: 0.9971061	total: 125ms	remaining: 2m 5s
1:	learn: 0.9205084	total: 258ms	remaining: 2m 8s
2:	learn: 0.8589815	total: 388ms	remaining: 2m 9s
3:	learn: 0.8092646	total: 529ms	remaining: 2m 11s
4:	learn: 0.7677051	total: 657ms	remaining: 2m 10s
5:	learn: 0.7326985	total: 790ms	remaining: 2m 10s
6:	learn: 0.7021389	total: 923ms	remaining: 2m 10s
7:	learn: 0.6765181	total: 1.06s	remaining: 2m 11s
8:	learn: 0.6533497	total: 1.19s	remaining: 2m 11s
9:	learn: 0.6333909	total: 1.37s	remaining: 2m 15s
10:	learn: 0.6160869	total: 1.5s	remaining: 2m 15s
11:	learn: 0.6011301	total: 1.63s	remaining: 2m 14s
12:	learn: 0.5881101	total: 1.76s	remaining: 2m 13s
13:	learn: 0.5759988	total: 1.9s	remaining: 2m 13s
14:	learn: 0.5654288	total: 2.03s	remaining: 2m 13s
15:	learn: 0.5552518	total: 2.18s	remaining: 2m 14s
16:	learn: 0.5467670	total: 2.32s	remaining: 2m 14s
17:	learn: 0.5396594	total: 2.46s	remaining: 2m 14s
18:	learn: 0.5326440	total: 2.6s	remaining: 2m 14

159:	learn: 0.4130408	total: 22.7s	remaining: 1m 59s
160:	learn: 0.4129306	total: 22.9s	remaining: 1m 59s
161:	learn: 0.4127112	total: 23s	remaining: 1m 58s
162:	learn: 0.4124757	total: 23.1s	remaining: 1m 58s
163:	learn: 0.4123373	total: 23.2s	remaining: 1m 58s
164:	learn: 0.4121743	total: 23.4s	remaining: 1m 58s
165:	learn: 0.4119830	total: 23.5s	remaining: 1m 58s
166:	learn: 0.4117142	total: 23.6s	remaining: 1m 57s
167:	learn: 0.4115699	total: 23.8s	remaining: 1m 57s
168:	learn: 0.4114598	total: 23.9s	remaining: 1m 57s
169:	learn: 0.4113432	total: 24s	remaining: 1m 57s
170:	learn: 0.4111626	total: 24.1s	remaining: 1m 57s
171:	learn: 0.4109264	total: 24.3s	remaining: 1m 56s
172:	learn: 0.4107320	total: 24.4s	remaining: 1m 56s
173:	learn: 0.4105844	total: 24.5s	remaining: 1m 56s
174:	learn: 0.4103704	total: 24.7s	remaining: 1m 56s
175:	learn: 0.4102306	total: 24.8s	remaining: 1m 56s
176:	learn: 0.4100598	total: 24.9s	remaining: 1m 55s
177:	learn: 0.4099716	total: 25.1s	remaining: 1m 5

316:	learn: 0.3971099	total: 45s	remaining: 1m 36s
317:	learn: 0.3970667	total: 45.1s	remaining: 1m 36s
318:	learn: 0.3969981	total: 45.3s	remaining: 1m 36s
319:	learn: 0.3969503	total: 45.4s	remaining: 1m 36s
320:	learn: 0.3969061	total: 45.5s	remaining: 1m 36s
321:	learn: 0.3968358	total: 45.7s	remaining: 1m 36s
322:	learn: 0.3967464	total: 45.8s	remaining: 1m 35s
323:	learn: 0.3966608	total: 46s	remaining: 1m 35s
324:	learn: 0.3966108	total: 46.1s	remaining: 1m 35s
325:	learn: 0.3965584	total: 46.2s	remaining: 1m 35s
326:	learn: 0.3964992	total: 46.4s	remaining: 1m 35s
327:	learn: 0.3964237	total: 46.5s	remaining: 1m 35s
328:	learn: 0.3963325	total: 46.6s	remaining: 1m 35s
329:	learn: 0.3962625	total: 46.8s	remaining: 1m 34s
330:	learn: 0.3962090	total: 46.9s	remaining: 1m 34s
331:	learn: 0.3961514	total: 47s	remaining: 1m 34s
332:	learn: 0.3961185	total: 47.2s	remaining: 1m 34s
333:	learn: 0.3960659	total: 47.4s	remaining: 1m 34s
334:	learn: 0.3960014	total: 47.5s	remaining: 1m 34s

472:	learn: 0.3899124	total: 1m 6s	remaining: 1m 14s
473:	learn: 0.3898844	total: 1m 6s	remaining: 1m 13s
474:	learn: 0.3898445	total: 1m 6s	remaining: 1m 13s
475:	learn: 0.3898283	total: 1m 6s	remaining: 1m 13s
476:	learn: 0.3897961	total: 1m 7s	remaining: 1m 13s
477:	learn: 0.3897519	total: 1m 7s	remaining: 1m 13s
478:	learn: 0.3897275	total: 1m 7s	remaining: 1m 13s
479:	learn: 0.3896852	total: 1m 7s	remaining: 1m 13s
480:	learn: 0.3896571	total: 1m 7s	remaining: 1m 12s
481:	learn: 0.3896244	total: 1m 7s	remaining: 1m 12s
482:	learn: 0.3895963	total: 1m 7s	remaining: 1m 12s
483:	learn: 0.3895360	total: 1m 8s	remaining: 1m 12s
484:	learn: 0.3895134	total: 1m 8s	remaining: 1m 12s
485:	learn: 0.3894691	total: 1m 8s	remaining: 1m 12s
486:	learn: 0.3894535	total: 1m 8s	remaining: 1m 12s
487:	learn: 0.3893890	total: 1m 8s	remaining: 1m 11s
488:	learn: 0.3893554	total: 1m 8s	remaining: 1m 11s
489:	learn: 0.3892916	total: 1m 8s	remaining: 1m 11s
490:	learn: 0.3892772	total: 1m 8s	remaining: 

628:	learn: 0.3853887	total: 1m 27s	remaining: 51.6s
629:	learn: 0.3853558	total: 1m 27s	remaining: 51.5s
630:	learn: 0.3853323	total: 1m 27s	remaining: 51.4s
631:	learn: 0.3853248	total: 1m 27s	remaining: 51.2s
632:	learn: 0.3853056	total: 1m 28s	remaining: 51.1s
633:	learn: 0.3852076	total: 1m 28s	remaining: 50.9s
634:	learn: 0.3851612	total: 1m 28s	remaining: 50.8s
635:	learn: 0.3851336	total: 1m 28s	remaining: 50.7s
636:	learn: 0.3851084	total: 1m 28s	remaining: 50.5s
637:	learn: 0.3850799	total: 1m 28s	remaining: 50.4s
638:	learn: 0.3850707	total: 1m 28s	remaining: 50.2s
639:	learn: 0.3850367	total: 1m 29s	remaining: 50.1s
640:	learn: 0.3850154	total: 1m 29s	remaining: 49.9s
641:	learn: 0.3850037	total: 1m 29s	remaining: 49.8s
642:	learn: 0.3849777	total: 1m 29s	remaining: 49.6s
643:	learn: 0.3849490	total: 1m 29s	remaining: 49.5s
644:	learn: 0.3849368	total: 1m 29s	remaining: 49.4s
645:	learn: 0.3849114	total: 1m 29s	remaining: 49.2s
646:	learn: 0.3848931	total: 1m 29s	remaining:

784:	learn: 0.3822439	total: 1m 49s	remaining: 30.1s
785:	learn: 0.3822219	total: 1m 49s	remaining: 29.9s
786:	learn: 0.3822049	total: 1m 50s	remaining: 29.8s
787:	learn: 0.3821897	total: 1m 50s	remaining: 29.6s
788:	learn: 0.3821751	total: 1m 50s	remaining: 29.5s
789:	learn: 0.3821608	total: 1m 50s	remaining: 29.4s
790:	learn: 0.3821287	total: 1m 50s	remaining: 29.2s
791:	learn: 0.3821153	total: 1m 50s	remaining: 29.1s
792:	learn: 0.3820785	total: 1m 50s	remaining: 28.9s
793:	learn: 0.3820551	total: 1m 50s	remaining: 28.8s
794:	learn: 0.3820448	total: 1m 51s	remaining: 28.6s
795:	learn: 0.3820208	total: 1m 51s	remaining: 28.5s
796:	learn: 0.3820112	total: 1m 51s	remaining: 28.4s
797:	learn: 0.3820057	total: 1m 51s	remaining: 28.2s
798:	learn: 0.3819862	total: 1m 51s	remaining: 28.1s
799:	learn: 0.3819496	total: 1m 51s	remaining: 27.9s
800:	learn: 0.3819386	total: 1m 51s	remaining: 27.8s
801:	learn: 0.3819204	total: 1m 51s	remaining: 27.6s
802:	learn: 0.3819130	total: 1m 52s	remaining:

941:	learn: 0.3799855	total: 2m 17s	remaining: 8.44s
942:	learn: 0.3799740	total: 2m 17s	remaining: 8.29s
943:	learn: 0.3799585	total: 2m 17s	remaining: 8.15s
944:	learn: 0.3799468	total: 2m 17s	remaining: 8s
945:	learn: 0.3799355	total: 2m 17s	remaining: 7.86s
946:	learn: 0.3799325	total: 2m 17s	remaining: 7.71s
947:	learn: 0.3799232	total: 2m 17s	remaining: 7.57s
948:	learn: 0.3799157	total: 2m 18s	remaining: 7.43s
949:	learn: 0.3799049	total: 2m 18s	remaining: 7.29s
950:	learn: 0.3798875	total: 2m 18s	remaining: 7.14s
951:	learn: 0.3798804	total: 2m 18s	remaining: 6.99s
952:	learn: 0.3798689	total: 2m 18s	remaining: 6.85s
953:	learn: 0.3798502	total: 2m 19s	remaining: 6.71s
954:	learn: 0.3798402	total: 2m 19s	remaining: 6.56s
955:	learn: 0.3798344	total: 2m 19s	remaining: 6.42s
956:	learn: 0.3798175	total: 2m 19s	remaining: 6.27s
957:	learn: 0.3798073	total: 2m 19s	remaining: 6.13s
958:	learn: 0.3797961	total: 2m 19s	remaining: 5.98s
959:	learn: 0.3797768	total: 2m 20s	remaining: 5.

In [54]:
company = 'МУП "КИНОВИДЕОСЕТЬ"'
extract_features(company, test=True)

array([[0.        , 0.04347826, 0.66666667, 0.33333333, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.5       , 0.1884058 , 0.46153846, 0.46153846, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [55]:
transform(company, clf.predict(extract_features(company, test=True)))

МУП
киновидеосеть
